# MIMIC Hyperparameter Tuning

This notebook runs through hyperparameter tuning for the internal MIMIC dataset. For this we use the Optuna library.

For users who have access to the MIMIC-III internal datasets - currently the notebook to generate the internal datasets is not uploaded. However, if you create a similar dataset to the one provided within the data guidance file using the MIMIC-III dataset then you will be able to run through this.

We validate our hyperparameter tuning results on our training dataset metrics - This is isn't optimal as usually it would be validated on a separate validation set. Hard to create an appropriate validation set in this instance as we would require the distributions for each variable column to look similar between training & validation.

NOTE: There are known limitations that are explained as they come up in these markdown cells.

In [ ]:
# Standard imports
from tokenize import String
import numpy as np
import pandas as pd
import torch

# VAE is in other folder
import sys
sys.path.append('../')

# Opacus support for differential privacy
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

# For VAE dataset formatting
from torch.utils.data import TensorDataset, DataLoader

# VAE functions
from VAE import Decoder, Encoder, VAE

# For datetime columns we need a transformer
from rdt.transformers import datetime

# Utility file contains all functions required to run notebook
from utils import mimic_pre_proc, constraint_filtering, plot_elbo, plot_likelihood_breakdown, plot_variable_distributions, reverse_transformers
from metrics import distribution_metrics

# Hyperparameter tuning library as well as pickle to save study objects
import optuna
import pickle

## Data Loading & Column Definitions

First we need to load in the MIMIC dataset from a specified filepath. 

We then need to create lists indicating which columns are:
a) continuous
b) categorical
c) datetime

Currently other data types are not supported. Importantly if columns contain missing data then they need to be dropped - Do not include these in original column lists & instead drop them from the loaded set in the cell below.

In [ ]:
# Load in the mimic single table data 

filepath = ""

data_supp = pd.read_csv(filepath)
# Save the original columns

original_categorical_columns = ['Categorical_1', 'Categorical_2', ...]
original_continuous_columns = ['Continuous_2', 'Continuous_2', ...]
original_datetime_columns = ['Datetime_1', 'Datetime_2', ...]
original_columns = original_categorical_columns + original_continuous_columns + original_datetime_columns

In [ ]:
# Drop columns that have missing data as these cannot be handled in the current implementation

data_supp = data_supp.drop('Missing_Column_1', axis = 1) # etc ...

## Data Pre-Processing

Data can be pre-processed in 2 ways. Either we use <b>"standard"</b> option which performs a standard scaler on continuous variables - This has known limitations as:

- Data in tables is usually non-gaussian and SynthVAE implements a gaussian loss, so this will perform worse unless the data is KNOWN to follow a gaussian distribution already.

Or we use the second option of <b>"GMM"</b>. This performs a variational gaussian mixture model to scale the data & transform it to a gaussian distribution. We use a maximum number of clusters of 10 but the variational method will select the best number of clusters for that continuous variable. This also has known limitations:

- 10 Clusters is arbitrary and may not be enough for certain variables.
- We are fitting a model to transform the data and hence we are approximating before model is trained. This will lose fidelity as the distribution will not be transformed perfectly.


For datasets that include datetime columns, original_metric_set returns the initial dataset after these columns have been transformed. This is because:

- Our evaluation suite cannot calculate certain metrics on datetime objects so these need to be converted to continuous values first

In [ ]:
pre_proc_method = "GMM" # Select pre-processing method standard or GMM

#%% -------- Data Pre-Processing -------- #

x_train, original_metric_set, reordered_dataframe_columns, continuous_transformers, categorical_transformers, datetime_transformers, num_categories, num_continuous = mimic_pre_proc(data_supp=data_supp, pre_proc_method=pre_proc_method)

## Creation & Training of VAE.

We can adapt certain parameters of the model e.g. batch size, latent dimension size etc. This model implements early stopping and these values can be adapted.

We can also activate differential privacy by implementing dp-sgd through the opacus library.

The user defined parameters are defined first and these are arbitrary. For example you could change batch size as well as other variables and if you wanted to do this then you simply move batch size into the objective function in the cell below and then follow the Optuna guidelines on creating a hyperparameter selection.

NOTE: training can be fail and cause errors if the hyperparameter values are not chosen carefully. In this example learning rate was left as <i>1e-3</i> rather than adapted as giving it a selection lead to errors in the training of the encoder - something to watch out for

In [ ]:
# User defined parameters

# General training
batch_size=32
n_epochs=5
logging_freq=1 # Number of epochs we should log the results to the user
patience=5 # How many epochs should we allow the model train to see if
# improvement is made
delta=10 # The difference between elbo values that registers an improvement
filepath=None # Where to save the best model


# Privacy params
differential_privacy = False # Do we want to implement differential privacy
sample_rate=0.1 # Sampling rate
noise_scale=None # Noise multiplier - influences how much noise to add
target_eps=1 # Target epsilon for privacy accountant
target_delta=1e-5 # Target delta for privacy accountant

# Define the metrics you want the model to evaluate

gower=False

# Prepare data for interaction with torch VAE
Y = torch.Tensor(x_train)
dataset = TensorDataset(Y)

generator = None
sample_rate = batch_size / len(dataset)
data_loader = DataLoader(
    dataset,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(dataset), sample_rate=sample_rate, generator=generator
    ),
    pin_memory=True,
    generator=generator,
)

# Setting Up Optuna Hyperparameter Tuning Objective Function

See markdown above for details

In [ ]:
# -------- Define our Optuna trial -------- #

def objective(trial, gower, differential_privacy=False, target_delta=1e-3, target_eps=10.0, n_epochs=50):

    latent_dim = trial.suggest_int('Latent Dimension', 2, 128, step=2) # Hyperparam
    hidden_dim = trial.suggest_int('Hidden Dimension', 32, 1024, step=32) # Hyperparam

    encoder = Encoder(x_train.shape[1], latent_dim, hidden_dim=hidden_dim)
    decoder = Decoder(
        latent_dim, num_continuous, num_categories=num_categories
    )

    lr = trial.suggest_float('Learning Rate', 1e-3, 1e-2, step=1e-5)
    vae = VAE(encoder, decoder, lr=1e-3) # lr hyperparam

    C = trial.suggest_int('C', 10, 1e4, step=50)

    if differential_privacy == True:
        log_elbo, log_reconstruction, log_divergence, log_categorical, log_numerical = vae.diff_priv_train(
            data_loader,
            n_epochs=n_epochs,
            C=C, # Hyperparam
            target_eps=target_eps,
            target_delta=target_delta, 
            sample_rate=sample_rate,
        )
        print(f"(epsilon, delta): {vae.get_privacy_spent(target_delta)}")

    else:

        log_elbo, log_reconstruction, log_divergence, log_categorical, log_numerical = vae.train(data_loader, n_epochs=n_epochs)

    # -------- Generate Synthetic Data -------- #

    synthetic_supp = constraint_filtering(
    n_rows=data_supp.shape[0], vae=vae, reordered_cols=reordered_dataframe_columns,
    data_supp_columns=data_supp.columns, cont_transformers=continuous_transformers,
    cat_transformers=categorical_transformers, date_transformers=datetime_transformers,
    pre_proc_method=pre_proc_method
    )

    # -------- Datetime Handling -------- #

    # If the dataset has datetimes then we need to re-convert these to a numerical
    # Value representing seconds, this is so we can evaluate the metrics on them

    metric_synthetic_supp = synthetic_supp.copy()

    for index, column in enumerate(original_datetime_columns):

        # Fit datetime transformer - converts to seconds
        temp_datetime = datetime.DatetimeTransformer()
        temp_datetime.fit(metric_synthetic_supp, columns = column)

        metric_synthetic_supp = temp_datetime.transform(metric_synthetic_supp)

    # -------- SDV Metrics -------- #
    # Calculate the sdv metrics for SynthVAE

    metrics = distribution_metrics(
    gower=gower, data_supp=data_supp, synthetic_supp=synthetic_supp,
    categorical_columns=original_categorical_columns, continuous_columns=original_continuous_columns,
    saving_filepath=None, pre_proc_method=pre_proc_method
    )

    # Optuna wants a list of values in float form

    list_metrics = [metrics[i] for i in metrics.columns]

    print(list_metrics)

    return list_metrics

## Hyperparameter Trials

Here we use optuna to set up a study and run it for a predefined number of trials. If the study has not already been created then set <b>first_run</b> to True. This will then create the study for running.

NOTE: directions show if we are maximising or minimising the metrics we are inputting. Most of SDV metrics require maximizing and that is why <b>directions</b> is set up like this. If you are inputting metrics that require minimizing then you need to set up your directions list accordingly.

In [ ]:
# If there is no study object in your folder then run and save the study so
# It can be resumed if needed

# User parameters

first_run=True  # First run indicates if we are creating a new hyperparam study
saving_filepath = "test.pkl" # To save the study if you wish - needs to be .pkl format
n_trials = 3 # Number of trials you want to hyperparameter tune for - needs to be .pkl format
loading_filepath=None # To load any older study if they have already been created

if(first_run==True):

    if(gower==True):
        directions = ['maximize' for i in range(6)]
    else:
        directions = ['maximize' for i in range(5)]

    study = optuna.create_study(directions=directions)

else:

    with open('{}'.format(loading_filepath), 'rb') as f:
        study = pickle.load(f)

study.optimize(
    lambda trial : objective(
    trial, gower=gower, differential_privacy=differential_privacy, target_delta=target_delta, target_eps=target_eps, n_epochs=n_epochs
    ), n_trials=n_trials, gc_after_trial=True
    ) # GC to avoid OOM

## Saving The Study

Here we use pickle to save the study so that it can be loaded up and run from its current point.

If your study is a multi objective study then it will give you multiple <b>best_trials</b> when you use <b>study.best_trials</b>. Depending your weighting for each metric, you can decide which study you will pick as optimal. An example of this is shown in the second cell where each metric is equally important and we average over them using the mean.

In [ ]:
#%% -------- Save The  Study -------- #

with open("{}".format(saving_filepath), 'wb') as f:
        pickle.dump(study, f)

In [ ]:
trial_averages = []

for trials in study.best_trials:

    metrics = trials.values
    trial_averages.append(np.mean(metrics))

# Now find the best trial

best_trial = np.argmax(np.asarray(trial_averages))

# Best trial hyperparameters

study.best_trials[best_trial].params